In [1]:
import sklearn
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import os
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
cd "/content/drive/MyDrive/money/data"

/content/drive/MyDrive/money/data


In [4]:
ls

10000dong/  20000dong/  50000dong/


In [5]:
# preprocessing
link = "/content/drive/MyDrive/money/data"
face_names = os.listdir(link)
face_path = []
for face_name in face_names:
  face_path.append(os.path.join(link, face_name))
print(face_names)
X = []
y = []
for i in range(len(face_path)):
  print(face_names[i])
  img_path = list(Path(face_path[i]).glob("*"))
  for j in img_path:
    img = plt.imread(j)
    img = cv2.resize(img, (128, 128))
    img.astype('float32')
    img = img/255
    X.append(img)
    y.append(i)

['50000dong', '20000dong', '10000dong']
50000dong
20000dong
10000dong


In [6]:
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)

(920, 128, 128, 3)
(920,)


In [7]:
from keras.utils import to_categorical
y = to_categorical(y)
print(y.shape)

(920, 3)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
from keras import models, layers
from keras.layers import Flatten, Dense, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils 
model = models.Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(128,128,3),padding = 'same'))
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu',padding = 'same'))
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',padding = 'same'))
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Flatten(input_shape=[128, 128, 3]))
model.add(Dense(1000, activation='relu'))
model.add(Dense(3000, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(face_names), activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 128)      0

In [11]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(X_train, y_train, epochs=400, batch_size=128)

Epoch 1/400
6/6 [==============================] - 12s 240ms/step - loss: 1.2698 - accuracy: 0.3967
Epoch 2/400
6/6 [==============================] - 1s 88ms/step - loss: 1.0947 - accuracy: 0.4511
Epoch 3/400
6/6 [==============================] - 1s 87ms/step - loss: 1.0782 - accuracy: 0.4511
Epoch 4/400
6/6 [==============================] - 1s 87ms/step - loss: 1.0788 - accuracy: 0.4511
Epoch 5/400
6/6 [==============================] - 1s 87ms/step - loss: 1.0734 - accuracy: 0.4511
Epoch 6/400
6/6 [==============================] - 1s 87ms/step - loss: 1.0725 - accuracy: 0.4511
Epoch 7/400
6/6 [==============================] - 1s 88ms/step - loss: 1.0684 - accuracy: 0.4511
Epoch 8/400
6/6 [==============================] - 1s 93ms/step - loss: 1.0621 - accuracy: 0.4511
Epoch 9/400
6/6 [==============================] - 1s 88ms/step - loss: 1.0676 - accuracy: 0.4511
Epoch 10/400
6/6 [==============================] - 1s 87ms/step - loss: 1.0679 - accuracy: 0.4511
Epoch 11/400
6/6 

In [24]:
n = 123
print(f'{n}/{len(y_test)}')
pred = model.predict(X_test[n].reshape((1, 128, 128, 3)))
print(f'Predict: {face_names[pred.argmax()]}  ({int(pred[0][pred.argmax()]*100)}%)')
print('True_label: ', face_names[y_test[n].argmax()])

123/184
1/1 [==============================] - 0s 28ms/step
Predict: 10000dong  (100%)
True_label:  10000dong
